<a href="https://colab.research.google.com/github/MarcelJonesRaymond/Neural-Network-and-Deep-Learning/blob/main/Week_2_of_mentoring_session(Fraud_Detection_Intro_to_NN_2)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Introduction to Neural Networks - Fraud Detection
Nilson reports that U.S. card fraud (credit, debt, etc) was reportedly  9𝑏𝑖𝑙𝑙𝑖𝑜𝑛𝑖𝑛2016𝑎𝑛𝑑𝑒𝑥𝑝𝑒𝑐𝑡𝑒𝑑𝑡𝑜𝑖𝑛𝑐𝑟𝑒𝑎𝑠𝑒𝑡𝑜 12 billion by 2020. For perspective, in 2017 both PayPal's and Mastercard's revenue was only $10.8 billion each.

Objective: In this notebook, given the credit card transactions, we will build a simple neural network (i.e., Multilayer perceptrons) for Fraud Detection using Keras.

In [52]:
import  pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras.layers import Flatten
from sklearn import metrics
from tensorflow.keras import optimizers

In [30]:
data=pd.read_csv('/content/creditcard.csv')

In [31]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [32]:
data.shape

(57531, 31)

In [33]:
data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
57526,47945,1.138100,0.186695,0.224979,1.098891,-0.149107,-0.494817,0.158646,-0.082146,0.031110,-0.033374,-0.234341,0.078180,-0.386803,0.498589,1.216187,-0.222097,-0.125371,-0.548831,-0.737141,-0.133280,0.070575,0.193698,-0.093600,0.078039,0.590292,-0.300950,0.021664,0.021820,36.86,0.0
57527,47946,-0.646913,1.130799,1.050571,-0.569363,0.844107,0.227605,0.808902,-0.130225,-0.290188,0.465887,0.360765,0.592729,1.073142,-0.241249,0.203440,0.635220,-1.311464,0.423780,0.792800,0.371398,-0.327854,-0.647464,-0.188020,-1.029697,0.014101,0.101051,0.084340,-0.074503,1.98,0.0
57528,47946,1.214047,0.217891,0.399390,0.945208,-0.101512,-0.108244,-0.099976,0.071885,0.248587,-0.053539,-0.672187,-0.208870,-0.667994,0.454367,1.449674,0.182817,-0.359178,-0.537780,-0.447389,-0.206486,-0.293180,-0.859018,0.125645,-0.508650,0.253563,-0.629445,0.040633,0.022467,2.99,0.0
57529,47946,-0.637264,1.187105,0.911446,0.827789,-0.241755,-0.431410,0.482341,0.395142,-1.095934,-0.306139,1.009983,0.671604,-0.062630,0.874290,0.280471,0.003488,-0.270139,0.667762,0.708641,-0.028700,0.272207,0.604289,-0.037705,0.339703,-0.250802,-0.350949,0.031500,0.094337,45.00,0.0
57530,47946,-1.054793,0.820159,0.709861,-0.071631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
data.sample(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
41798,40835,-7.276606,-8.175352,-0.293301,0.064510,0.540631,-1.169022,1.155819,0.715566,-0.509314,-1.980219,0.889027,0.937406,0.633233,0.685253,0.031555,1.438474,-0.904970,0.525446,-1.916218,4.000910,1.325339,-0.295320,2.301546,-0.321004,0.714184,1.156191,-0.260645,-0.784639,1131.81,0.0
14374,25493,1.319830,0.280201,0.177130,0.417199,-0.007772,-0.496260,0.092996,-0.192938,-0.006769,-0.085675,-0.842308,0.476437,1.183295,0.045459,1.013968,0.513340,-0.768874,-0.357452,0.192048,-0.019893,-0.304245,-0.839654,0.024228,-0.440574,0.353706,0.144149,-0.025438,0.012070,1.79,0.0
20540,31112,1.082511,0.039081,0.348632,1.431779,-0.230045,-0.175780,0.093136,0.000083,0.355735,-0.130903,-0.614517,0.286046,-0.584829,0.167166,-0.038013,-0.695909,0.291214,-0.912921,-0.378138,-0.133436,-0.081423,-0.116552,-0.093796,0.084413,0.632043,-0.315787,0.028850,0.022434,50.80,0.0
18193,29278,1.097353,-0.064921,0.067244,0.575795,-0.299773,-0.529791,0.079116,0.055554,-0.108806,0.139803,1.478069,0.065103,-1.929261,1.026747,0.534146,-0.063685,-0.079644,-0.272916,-0.259276,-0.167191,0.040327,-0.040441,-0.034003,0.212011,0.361252,0.368735,-0.059096,-0.001931,48.03,0.0
9006,12503,1.185166,0.173280,0.595597,1.410592,-0.056552,0.318562,-0.271888,0.015297,1.914314,-0.594435,-0.115871,-2.066351,1.634463,1.275495,-0.849272,-0.597885,0.835593,-0.407396,-0.357719,-0.212245,-0.234098,-0.136251,-0.126911,-0.468402,0.680386,-0.292450,0.030697,0.008552,8.79,0.0


In [35]:
data.drop('Time',axis=1,inplace=True)

In [37]:
data.isna().sum()

V1        0
V2        0
V3        0
V4        0
V5        1
V6        1
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [38]:
data.dropna(axis=0,inplace=True)

In [40]:
data.shape

(57530, 30)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57530 entries, 0 to 57529
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      57530 non-null  float64
 1   V2      57530 non-null  float64
 2   V3      57530 non-null  float64
 3   V4      57530 non-null  float64
 4   V5      57530 non-null  float64
 5   V6      57530 non-null  float64
 6   V7      57530 non-null  float64
 7   V8      57530 non-null  float64
 8   V9      57530 non-null  float64
 9   V10     57530 non-null  float64
 10  V11     57530 non-null  float64
 11  V12     57530 non-null  float64
 12  V13     57530 non-null  float64
 13  V14     57530 non-null  float64
 14  V15     57530 non-null  float64
 15  V16     57530 non-null  float64
 16  V17     57530 non-null  float64
 17  V18     57530 non-null  float64
 18  V19     57530 non-null  float64
 19  V20     57530 non-null  float64
 20  V21     57530 non-null  float64
 21  V22     57530 non-null  float64
 22

In [42]:
data.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
V1,57530.0,-0.238414,1.866763,-56.407510,-0.994241,-0.244532,1.155072,1.960497
V2,57530.0,-0.008017,1.643765,-72.715728,-0.582023,0.074606,0.731609,18.183626
V3,57530.0,0.690217,1.456793,-32.965346,0.207683,0.783075,1.417420,4.101716
V4,57530.0,0.171639,1.387939,-5.172595,-0.727951,0.182415,1.054258,16.491217
V5,57530.0,-0.259999,1.399214,-42.147898,-0.877906,-0.293590,0.279316,34.801666
V6,57530.0,0.102886,1.309931,-26.160506,-0.635432,-0.152452,0.491946,22.529298
V7,57530.0,-0.117290,1.257196,-26.548144,-0.606257,-0.075688,0.424691,36.677268
V8,57530.0,0.056047,1.195360,-41.484823,-0.145380,0.061320,0.335839,20.007208
V9,57530.0,0.072816,1.188572,-9.283925,-0.642709,-0.032749,0.751398,10.392889
V10,57530.0,-0.047614,1.113864,-18.271168,-0.521690,-0.104014,0.428848,13.198226


In [46]:
data.corr().style.background_gradient('magma')

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
V1,1.000000,-0.013304,0.275213,-0.091369,0.055565,0.101968,0.146176,-0.080813,-0.035401,0.058554,-0.022999,0.037409,0.006841,0.112750,0.050551,0.091471,0.102530,0.018915,0.001977,-0.060100,-0.049117,-0.032571,-0.079545,-0.009512,0.179517,0.023898,-0.056255,0.098180,-0.243385,-0.196402
V2,-0.013304,1.000000,-0.164294,0.086976,-0.100905,0.004535,-0.044952,0.073656,-0.015266,-0.025245,0.065481,-0.099976,0.052298,-0.046162,0.052047,-0.034686,-0.087086,-0.027418,0.005189,-0.126439,-0.004487,-0.045486,-0.006052,-0.017847,-0.090413,-0.032205,0.046455,-0.059896,-0.549047,0.166094
V3,0.275213,-0.164294,1.000000,-0.128218,0.274981,-0.021883,0.304326,-0.250145,0.157525,0.191330,-0.115279,0.123442,-0.026665,0.179353,-0.170751,0.015702,0.180334,0.042199,-0.043643,-0.102250,0.000204,0.238993,0.052348,0.020302,-0.184142,0.048422,-0.108851,0.036010,-0.201460,-0.377094
V4,-0.091369,0.086976,-0.128218,1.000000,-0.033956,-0.054469,-0.077645,0.082040,0.011856,-0.111233,0.032629,-0.050997,0.008723,-0.053226,-0.077811,-0.130140,-0.032745,-0.049554,-0.005017,0.034959,0.017012,0.015033,-0.009560,-0.006195,-0.007400,0.006405,0.053996,0.005474,0.091140,0.207031
V5,0.055565,-0.100905,0.274981,-0.033956,1.000000,0.065827,0.027896,-0.095800,0.064382,0.137901,-0.045394,0.048361,0.047153,0.080296,0.086138,0.134893,0.048186,0.084328,0.008406,-0.056283,-0.071731,-0.062612,0.024650,-0.016122,-0.024175,-0.038636,-0.058984,0.035366,-0.377999,-0.190523
V6,0.101968,0.004535,-0.021883,-0.054469,0.065827,1.000000,0.100457,-0.045775,0.046973,0.022053,-0.101133,0.019622,0.002851,0.049405,-0.114659,0.002044,0.026869,0.050557,0.085234,0.034803,0.038030,0.023848,-0.019793,0.008921,0.051236,0.011447,-0.017326,-0.037396,0.204797,-0.091866
V7,0.146176,-0.044952,0.304326,-0.077645,0.027896,0.100457,1.000000,-0.138216,0.065372,0.138108,-0.078909,0.147524,-0.006529,0.094385,0.068543,0.133990,0.141889,0.099309,-0.060236,0.080033,-0.030477,-0.020306,0.075942,-0.003238,-0.110876,-0.041290,-0.128472,-0.052195,0.361357,-0.313196
V8,-0.080813,0.073656,-0.250145,0.082040,-0.095800,-0.045775,-0.138216,1.000000,-0.067124,-0.129913,0.012892,-0.061400,-0.015007,-0.056224,-0.038899,-0.072036,-0.080024,-0.038994,0.025948,0.035615,-0.037629,0.038889,-0.024534,0.006354,0.022128,0.002016,0.027752,0.046522,-0.091436,0.151449
V9,-0.035401,-0.015266,0.157525,0.011856,0.064382,0.046973,0.065372,-0.067124,1.000000,-0.040741,0.035751,-0.144137,0.124655,0.192301,-0.170779,-0.035518,0.189133,0.050250,-0.017785,0.007091,-0.020797,0.026527,-0.040503,0.008725,0.106211,0.100203,-0.030780,-0.024827,-0.020796,-0.154176
V10,0.058554,-0.025245,0.191330,-0.111233,0.137901,0.022053,0.138108,-0.129913,-0.040741,1.000000,-0.077015,0.150678,-0.060981,0.033170,0.066123,0.166178,0.199966,0.087342,-0.009523,-0.025289,-0.031381,-0.036372,0.000597,0.008065,0.001926,-0.016744,-0.084649,-0.018499,-0.127023,-0.337047


Features vs Target

In [49]:
x=data.iloc[:,:-1]
y=data[['Class']]

In [50]:
x.shape

(57530, 29)

In [51]:
y.shape

(57530, 1)

In [53]:
X=preprocessing.normalize(x)

In [57]:
X=pd.DataFrame(X,columns=x.columns)

In [58]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

In [59]:
model=Sequential()
model.add(Dense(64, input_shape=(29,) ,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [60]:
sgd=optimizers.Adam(lr=0.001)
model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1920      
                                                                 
 batch_normalization (BatchN  (None, 64)               256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,241
Trainable params: 2,113
Non-trainable params: 128
_________________________________________________________________


In [62]:
model.fit(x_train,y_train,epochs=10,batch_size=700,verbose=1)

Epoch 1/10
58/58 [==============================] - 1s 3ms/step - loss: 0.6950 - accuracy: 0.7337
Epoch 2/10
58/58 [==============================] - 0s 3ms/step - loss: 0.4504 - accuracy: 0.9175
Epoch 3/10
58/58 [==============================] - 0s 3ms/step - loss: 0.2708 - accuracy: 0.9680
Epoch 4/10
58/58 [==============================] - 0s 3ms/step - loss: 0.1526 - accuracy: 0.9857
Epoch 5/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0928 - accuracy: 0.9916
Epoch 6/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0605 - accuracy: 0.9952
Epoch 7/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0397 - accuracy: 0.9961
Epoch 8/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0333 - accuracy: 0.9966
Epoch 9/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0265 - accuracy: 0.9970
Epoch 10/10
58/58 [==============================] - 0s 3ms/step - loss: 0.0254 - accuracy: 0.9971


In [63]:
model.evaluate(x_test,y_test)

540/540 [==============================] - 1s 1ms/step - loss: 0.0143 - accuracy: 0.9986


[0.014331687241792679, 0.9985514879226685]